In [1]:
using Pkg;
Pkg.activate(".");
Pkg.instantiate();

  Activating project at `~/Repos/DoingRightNow-Analysis`


In [1]:
using DataFrames, Arrow, CategoricalArrays, ScientificTypes, MLJ

In [2]:
DATA_FILE_PATH = "./data/model_data.arrow";
df = DataFrame(Arrow.Table(DATA_FILE_PATH));
df = copy(df)

Row,TUCASEID,TUACTIVITY_N,TUSTARTTIM,TUSTOPTIME,start_time_int,stop_time_int,TRTIER2,snap_time_int,TULINENO,GESTFIPS_label,HEFAMINC_label,PEMARITL_label,HETENURE_label,PRTAGE,TUDIARYDAY,TUDIARYDAY_label
,Int64,Int64,Time,Time,Int64,Int64,Int64,Int64,Int64?,String?,String?,String?,String,Int64?,Int64?,String?
1,20210101210037,1,04:00:00,06:00:00,0,120,101,0,1,CO,"100,000 to 149,999",Not Married,Own,60,5,Thursday
2,20210101210081,1,04:00:00,09:00:00,0,300,101,0,1,IL,"35,000 to 39,999",Not Married,Own,30,7,Saturday
3,20210101210099,1,04:00:00,06:00:00,0,120,101,0,1,AZ,"100,000 to 149,999",Married,Own,43,5,Thursday
4,20210101210101,1,04:00:00,10:00:00,0,360,101,0,1,OH,"35,000 to 39,999",Not Married,Rent,23,6,Friday
5,20210101210109,1,04:00:00,11:00:00,0,420,101,0,1,GA,"12,500 to 14,999",Not Married,Rent,20,7,Saturday
6,20210101210115,1,04:00:00,12:00:00,0,480,101,0,1,FL,"10,000 to 12,499",Not Married,Rent,15,1,Sunday
7,20210101210131,1,04:00:00,07:30:00,0,210,101,0,1,OH,"50,000 to 59,999",Not Married,Own,24,1,Sunday
8,20210101210139,1,04:00:00,09:00:00,0,300,101,0,1,IA,"100,000 to 149,999",Not Married,Own,16,1,Sunday
9,20210101210155,1,04:00:00,12:00:00,0,480,101,0,1,TX,"35,000 to 39,999",Not Married,Own,15,1,Sunday


In [3]:

function clean_data!(df)
    
    # Fix machine types.
    HEFAMINC_ordered_set = [
        "Less than 5,000",
        "5,000 to 7,499",
        "7,500 to 9,999",
        "10,000 to 12,499",
        "12,500 to 14,999",
        "15,000 to 19,999",
        "20,000 to 24,999",
        "25,000 to 29,999",
        "30,000 to 34,999",
        "35,000 to 39,999",
        "40,000 to 49,999",
        "50,000 to 59,999",
        "60,000 to 74,999",
        "75,000 to 99,999",
        "100,000 to 149,999",
        "150,000 and over"
    ]

    df.TRTIER2 = categorical(df.TRTIER2)
    df.GESTFIPS_label = categorical(df.GESTFIPS_label)
    df.HEFAMINC_label = categorical(df.HEFAMINC_label; levels=HEFAMINC_ordered_set, ordered=true)
    df.PEMARITL_label = categorical(df.PEMARITL_label)
    df.HETENURE_label = categorical(df.HETENURE_label)
    df.TUDIARYDAY_label = categorical(df.TUDIARYDAY_label)

    # drop columns and disallow missing.
    drop_cols = [
        :TUCASEID,:TUACTIVITY_N,:TUSTARTTIM,:TUSTOPTIME,
        :start_time_int,:stop_time_int,:TULINENO, :TUDIARYDAY
        ]
    select!(df, Not(drop_cols))
    disallowmissing!(df)

    # Define scientific types.
    coerce!(df, :snap_time_int => Continuous, :PRTAGE => Continuous)
end

clean_data!(df);

In [4]:
y, X = unpack(df, ==(:TRTIER2));

## Find the right model to use

We'll take a look at what type of models are available to MLJ to predict on our target.

In [5]:
for m in models(matching(X,y))
    if m.prediction_type == :probabilistic
        println(rpad(m.name, 30), "($(m.package_name))")
    end
end

CatBoostClassifier            (CatBoost)
ConstantClassifier            (MLJModels)
DecisionTreeClassifier        (BetaML)
RandomForestClassifier        (BetaML)


The only models showing are tree-based models. We're prodicting a multi-class category. And this is how it is encoded in the data. Tree-based models will handle this explicitly.

But we _should_ be able to use something like a multivariate logistic regression, shouldn't we? Likely, the reason is typing. A regression isn't going to work on non-encoded predictors. According to the documentation, it _should_ properly interpret the multivariate target though.

In [6]:
# One Hot Encode X into a new object called X2.
ohe = OneHotEncoder(drop_last=true)
mach = fit!(machine(ohe, X))
X2 = MLJ.transform(mach, X)

# Search for the available models.
for m in models(matching(X2,y))
    if m.prediction_type == :probabilistic
        println(rpad(m.name, 30), "($(m.package_name))")
    end
end

┌ Info: Training machine(OneHotEncoder(features = Symbol[], …), …).
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/g5E7V/src/machines.jl:492
┌ Info: Spawning 50 sub-features to one-hot encode feature :GESTFIPS_label.
└ @ MLJModels /Users/mph/.julia/packages/MLJModels/UM8fF/src/builtins/Transformers.jl:878


┌ Info: Spawning 15 sub-features to one-hot encode feature :HEFAMINC_label.
└ @ MLJModels /Users/mph/.julia/packages/MLJModels/UM8fF/src/builtins/Transformers.jl:878
┌ Info: Spawning 1 sub-features to one-hot encode feature :PEMARITL_label.
└ @ MLJModels /Users/mph/.julia/packages/MLJModels/UM8fF/src/builtins/Transformers.jl:878
┌ Info: Spawning 2 sub-features to one-hot encode feature :HETENURE_label.
└ @ MLJModels /Users/mph/.julia/packages/MLJModels/UM8fF/src/builtins/Transformers.jl:878
┌ Info: Spawning 6 sub-features to one-hot encode feature :TUDIARYDAY_label.
└ @ MLJModels /Users/mph/.julia/packages/MLJModels/UM8fF/src/builtins/Transformers.jl:878


AdaBoostClassifier            (MLJScikitLearnInterface)
AdaBoostStumpClassifier       (DecisionTree)
BaggingClassifier             (MLJScikitLearnInterface)
BayesianLDA                   (MLJScikitLearnInterface)
BayesianLDA                   (MultivariateStats)
BayesianQDA                   (MLJScikitLearnInterface)
BayesianSubspaceLDA           (MultivariateStats)
CatBoostClassifier            (CatBoost)
ConstantClassifier            (MLJModels)
DecisionTreeClassifier        (BetaML)
DecisionTreeClassifier        (DecisionTree)
DummyClassifier               (MLJScikitLearnInterface)
EvoTreeClassifier             (EvoTrees)
ExtraTreesClassifier          (MLJScikitLearnInterface)
GaussianNBClassifier          (MLJScikitLearnInterface)
GaussianNBClassifier          (NaiveBayes)
GaussianProcessClassifier     (MLJScikitLearnInterface)
GradientBoostingClassifier    (MLJScikitLearnInterface)
KNNClassifier                 (NearestNeighborModels)
KNeighborsClassifier          (MLJScikitLearnI

That's a big variety of models to choose from.

We'll start from the smaller list of tree-based models. The random forest is a good one. We can do this two ways -- by using the default `RandomForestClassifier` or by composing our own bagging of a set of `DecisionTreeClassifier` models.

The easy, fast thing to do would be to use the default. But I'd like to get some practice in. So I'm going to do the bagging from scratch.

Before I continue, I'm going to partition the data into testing and training.

In [7]:
train, test = partition(eachindex(y), 0.8)

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  2093636, 2093637, 2093638, 2093639, 2093640, 2093641, 2093642, 2093643, 2093644, 2093645], [2093646, 2093647, 2093648, 2093649, 2093650, 2093651, 2093652, 2093653, 2093654, 2093655  …  2617047, 2617048, 2617049, 2617050, 2617051, 2617052, 2617053, 2617054, 2617055, 2617056])

## Random Forest Classifier

Note, a lot of this is adopted from [this tutorial](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/ensembles-2/).

We'll start with a single tree.

The `DecisionTreeClassifier` from the `BetaML` package works with no encoding or transformation. But it takes a very long time to run. We'll try setting up a pipeline to transform the data ard run the `DecisionTreeClassifier` from the `DecisionTree` package.

In [46]:
# Load models from packages.
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/mph/.julia/packages/MLJModels/UM8fF/src/loading.jl:159


MLJDecisionTreeInterface.DecisionTreeClassifier

### Step 0 -- Use `EnsembleModel` to roll our own random forest

ProbabilisticEnsembleModel(
  model = DecisionTreeClassifier(
        max_depth = -1, 
        min_samples_leaf = 1, 
        min_samples_split = 2, 
        min_purity_increase = 0.0, 
        n_subfeatures = 0, 
        post_prune = false, 
        merge_purity_threshold = 1.0, 
        display_depth = 5, 
        feature_importance = :impurity, 
        rng = Random._GLOBAL_RNG()), 
  atomic_weights = Float64[], 
  bagging_fraction = 0.8, 
  rng = Random._GLOBAL_RNG(), 
  n = 20, 
  acceleration = CPU1{Nothing}(nothing), 
  out_of_bag_measure = Any[])

### Step 1 -- Define a new model struct.

Likely this is a probabalistic model. We'll need to confirm this and define a probabalistic network composite model.

In [52]:
supertype(typeof(DecisionTreeClassifier()))    # Should be "Probabilistic"

Probabilistic

In [59]:
# Define a new model struct.
mutable struct CompositeA <: ProbabilisticNetworkComposite
    preprocessor    # This part does the pre-processing.
    classifier    # This part does the classifying
end

### Step 2 -- Create and wrap the learning network in `prefit`

In [64]:
# Wrap the above steps into a function called `prefit`
import MLJBase    # We need to import in order to overload `MLJBase.prefit`
function MLJBase.prefit(composite::CompositeA, verbosity, X, y)
    # Define data input nodes. We just want the training set.
    Xs = source(X[train,:])
    ys = source(y[train])

    # First machine -- We substitute the symbols in the struct defined above for the model objects.
    mach1 = machine(:preprocessor, Xs)
    x = MLJ.transform(mach1, Xs)    # `transform` has duplicated namespace. So we specify `MLJ.transform`
    mach2 = machine(:classifier, x, ys)
    ŷ = predict(mach2, x)

    verbosity > 0 && @info "I'm a noisy fellow!"

    #return "learning network interface":
    return (; predict=ŷ)
end

`prefit` always returns a _learning network interface_. Here, the inteface dictates that calling `predict(mach, Xnew)` on a machine `mach` bound to some instance of `CompositeA` should internally call `y\hat(Xnew)`.


This means we can use the above like any other model.

In [91]:
info(tree)

(name = "DecisionTreeClassifier",
 package_name = "DecisionTree",
 is_supervised = true,
 abstract_type = Probabilistic,
 deep_properties = (),
 docstring = "```\nDecisionTreeClassifier\n```\n\nA model type for c...",
 fit_data_scitype =
     Tuple{Table{<:Union{AbstractVector{<:Continuous}, AbstractVector{<:Count}, AbstractVector{<:OrderedFactor}}}, AbstractVector{<:Finite}},
 human_name = "CART decision tree classifier",
 hyperparameter_ranges = (nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing,
                          nothing),
 hyperparameter_types = ("Int64",
                         "Int64",
                         "Int64",
                         "Float64",
                         "Int64",
                         "Bool",
                 

In [104]:
using MLJ

one_hot_encoder = OneHotEncoder()
tree = DecisionTreeClassifier(n_subfeatures=3)
ensemble_model = EnsembleModel(model=tree, n=20)

composite_a = CompositeA(one_hot_encoder,ensemble_model)

CompositeA(
  preprocessor = OneHotEncoder(
        features = Symbol[], 
        drop_last = false, 
        ordered_factor = true, 
        ignore = false), 
  classifier = ProbabilisticEnsembleModel(
        model = DecisionTreeClassifier(max_depth = -1, …), 
        atomic_weights = Float64[], 
        bagging_fraction = 0.8, 
        rng = Random._GLOBAL_RNG(), 
        n = 20, 
        acceleration = CPU1{Nothing}(nothing), 
        out_of_bag_measure = Any[]))

In [109]:
mach = machine(composite_a, X, y)
#fit!(mach, rows=train, verbosity=0)
estimates = evaluate!(mach, measure=cross_entropy)    # Equal to fit! then predict! then calling the measure.

Evaluating over 6 folds:  33%[========>                ]  ETA: 0:13:07

Evaluating over 6 folds:  50%[============>            ]  ETA: 0:10:11

Evaluating over 6 folds:  67%[================>        ]  ETA: 0:06:59

Evaluating over 6 folds:  83%[====================>    ]  ETA: 0:03:35

Evaluating over 6 folds: 100%[=========================] Time: 0:22:30


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────────────────────────┬───────────┬─────────────┬─────────┬──────────
│ measure                        │ operation │ measurement │ 1.96*SE │ per_fol ⋯
├────────────────────────────────┼───────────┼─────────────┼─────────┼──────────
│ LogLoss(                       │ predict   │ 15.3        │ 4.56    │ [21.4,  ⋯
│   tol = 2.220446049250313e-16) │           │             │         │         ⋯
└────────────────────────────────┴───────────┴─────────────┴─────────┴──────────
                                                                1 column omitted


### Tuning Hyperparameters

Let's start by tuning the `tree.n_subfeatures` parameter.

In [112]:
mach.model.classifier.model.n_subfeatures

3

In [ ]:
composite_a

In [116]:
r_n_subfeatures = range(composite_a, :(classifier.model.n_subfeatures),lower=1, upper=6)
tuned_composite_a = TunedModel(
    composite_a,
    range=r_n_subfeatures,
    tuning=RandomSearch(rng=123),
    measure=cross_entropy,
    resampling=CV(nfolds=6),
    n=100,
)
mach = machine(tuned_composite_a, X, y) |> fit!
report(mach).best_model
# estimates2 = evaluate!(mach, measure=cross_entropy)    # Equal to fit! then predict! then calling the measure.

┌ Info: Training machine(ProbabilisticTunedModel(model = CompositeA(preprocessor = OneHotEncoder(features = Symbol[], …), …), …), …).
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/g5E7V/src/machines.jl:492
┌ Info: Attempting to evaluate 100 models.
└ @ MLJTuning /Users/mph/.julia/packages/MLJTuning/ZFg3R/src/tuned_models.jl:727
Evaluating over 100 metamodels:   0%[>                        ]  ETA: N/A